playlist_processing.ipynb

cleaned_song_meta.json(음악 메타 데이터), train.json(플레이리스트)를 사용해
플레이리스트에 존재하는 태그들을 song_id에 매핑한다.이 때 태그는 가장 많이 나온 20개의 태그만 저장한다.

count는 플레이리스트에 얼마나 많이 등장했는지 weight를 주기 위해 저장한다.

In [25]:
import json
import pandas as pd

"""
Method
"""

# Load cleaned song meta data(79,814)
def load_song_meta() -> pd.DataFrame:
    with open('./cleaned_song_meta.json', 'r', encoding='UTF8') as f:
        data = json.load(f)
    df = pd.json_normalize(data)
    df = df[['id', 'song_name', 'artist_name_basket']]
    df['tags'] = [[] for _ in range(len(df))]
    df['count'] = 0
    return df

# Load playlist data(115,071)
def load_playlist() -> pd.DataFrame:
    with open('./train.json', 'r', encoding='UTF8') as f:
        data = json.load(f)
    df = pd.json_normalize(data)
    df.sort_values(by=['like_cnt', 'updt_date'], ascending=False, inplace=True)
    return df

# Add playlist's @tags to each songs and update @count to give weights.
def processing_playlist(song_df: pd.DataFrame, playlist_df: pd.DataFrame) -> pd.DataFrame:
    for idx_, row in playlist_df.iterrows():
        cnt = row.like_cnt
        song_list = row.songs
        tags = row.tags
        for song in song_list:
            if song in song_df['id'].values:
                idx, = song_df.index[song_df['id'] == song].values
                song_df.at[idx, 'count'] += 1
                origin_tags = dict(song_df.at[idx, 'tags'])
                
                for tag in tags:
                    if tag not in origin_tags: origin_tags[tag] = 1
                    elif tag in origin_tags: origin_tags[tag] += 1
                
                if len(origin_tags) > 20:
                    new_dict = {}
                    for tag, cnt in sorted(list(origin_tags.items()), key=lambda x: -x[1])[:20]:
                        new_dict[tag] = cnt
                    song_df.at[idx, 'tags'] = list(new_dict.items())
                else:
                    song_df.at[idx, 'tags'] = list(origin_tags.items())
    
    for idx, row in song_df.iterrows():    
        song_df.at[idx, 'tags'] = list(dict(row.tags).keys())
    song_df.sort_values(by='count', ascending=False, inplace=True)
    return song_df
        

In [26]:
processed_song = processing_playlist(load_song_meta(), load_playlist())

In [28]:
processed_song
    

,id,song_name,artist_name_basket,tags,count
16162,144663,밤편지,[아이유],"[감성, 발라드, 새벽, 밤, 인디, 비오는날, 슬픔, 새벽감성, 여름밤, 감성발라...",2175
13048,116573,안아줘,[정준일],"[발라드, 이별, 새벽, 감성, 밤, 슬픔, 힐링, 기분전환, 가을, 카페, 비오는...",2121
40220,357367,비,[폴킴],"[감성, 발라드, 비오는날, 이별, 새벽, 사랑, 비, 휴식, 밤, 힐링, 센치, ...",1981
41312,366786,가끔 미치도록 네가 안고 싶어질 때가 있어,[가을방학],"[새벽, 인디, 밤, 감성, 발라드, 힐링, 추억, 기분전환, 카페, 비오는날, 인...",1919
14889,133143,"그대와 나, 설레임 (Feat. 소울맨)",[어쿠스틱 콜라보],"[기분전환, 힐링, 카페, 발라드, 감성, 새벽, 밤, 기분좋은, 인디음악, 설레임...",1633
...,...,...,...,...,...
35847,318071,자화상,[미기],[],0
12188,108886,Techno Shoes,[텔레파시],[],0
12191,108913,사랑의 굴레,[이봉근],[],0
35832,317919,ㅠ,[한국사람],[],0


In [29]:
processed_song.to_json('processed_song.json', orient='records')